In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
import torch
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
login(token="")

/ext3/miniconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/rl3725/.cache/huggingface/token
Login successful


In [3]:
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="meta-llama/Llama-2-7b-chat-hf", local_dir="./data/7b-chat")

## 1. Zero-Shot Prompting

In [6]:
tokenizer = AutoTokenizer.from_pretrained("/vast/work/public/ml-datasets/llama-2/Llama-2-7b-hf", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("/vast/work/public/ml-datasets/llama-2/Llama-2-7b-hf", device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [02:23<00:00, 71.88s/it]


In [3]:
prompt = """
<s>[INST] <<SYS>>
You are a reasoning system that has to identify in which step of a recipe the user is on given some inputs from a perception system.<</SYS>>

Can you help me? [/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
Your are a reasoning system that has to identify in which step of a recipe the user is on given some inputs from a perception system.<</SYS>>

Can you help me? [/INST]

Of course! I'd be happy to help you identify the step of a recipe that a user is on based on inputs from a perception system. Can you please provide me with more details on the following:

1. The recipe: Can you provide me with the name of the recipe or a brief description of it?
2. The perception system: How does the perception system provide inputs to the reasoning system? For example, are the inputs in the form of text, images, or audio?
3. The user's inputs: What are the user's inputs in the recipe? For example, are they selecting ingredients, following a list of instructions, or providing feedback on the cooking process?

Once I have this information, I can develop a reasoning system that can accurately identify the step of the recipe that the user is on based on the inputs from the 

In [4]:
prompt = """
<s>[INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.,
2: Add 1/2 cup water to the bowl.,
3: Microwave for 30 seconds.,
4: Add handful of raisins and stir.,
5: Microwave for 60 more seconds then stir.,
6: Top with banana slices.,
7: Sprinkle with cinnamon and drizzle with honey before serving.

And these are the possible states detected by the perception system and their probabilities:
oatmeal with water:0.54,
oatmeal with raisins with banana with cinnamon:0.06,
oatmeal with raisins:0.13,
oatmeal with raisins with banana:0.01,
oatmeal[dry]:0.15,
oatmeal with raisins with banana with cinnamon with honey:0.06,
cooked oatmeal:0.00,
empty:0.01 <</SYS>>
So, in which step of the recipe the user is?
[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
</s><s> [INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.,
2: Add 1/2 cup water to the bowl.,
3: Microwave for 30 seconds.,
4: Add handful of raisins and stir.,
5: Microwave for 60 more seconds then stir.,
6: Top with banana slices.,
7: Sprinkle with cinnamon and drizzle with honey before serving.

And these are the possible states detected by the perception system and their probabilities:
oatmeal with water:0.54,
oatmeal with raisins with banana with cinnamon:0.06,
oatmeal with raisins:0.13,
oatmeal with raisins with banana:0.01,
oatmeal[dry]:0.15,
oatmeal with raisins with banana with cinnamon with honey:0.06,
cooked oatmeal:0.00,
empty:0.01 <</SYS>>
So, in which step of the recipe the user is?
[/INST]
Based on the information provided, the user is most likely in step 3 of the recipe, as the probabilities of the possible states detected by the perception system indicate that the 

#### Adding Information about the States

In [8]:
tokenizer = AutoTokenizer.from_pretrained("./data/7b-chat", use_fast=True)
model = AutoModelForCausalLM.from_pretrained("./data/7b-chat", device_map="auto")

In [2]:
prompt = """
<s>[INST] <<SYS>>
Your are a reasoning system that has to identify in which step of a recipe the user is on given some inputs from a perception system.<</SYS>>

Can you help me? [/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
Your are a reasoning system that has to identify in which step of a recipe the user is on given some inputs from a perception system.<</SYS>>

Can you help me? [/INST]

Of course! I'd be happy to help you identify the step of a recipe that a user is on based on inputs from a perception system. Can you please provide me with more details about the recipe and the inputs from the perception system? For example:

* What is the name of the recipe?
* What are the steps involved in the recipe?
* What are the inputs from the perception system (e.g. images, text, audio)?
* How does the perception system provide the inputs (e.g. through a web interface, a mobile app, a voice assistant)?

Once I have this information, I can help you develop a reasoning system that can accurately identify the step of the recipe that the user is on based on the inputs from the perception system.</s>


In [4]:
prompt = """
<s>[INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.,
2: Add 1/2 cup water to the bowl.,
3: Microwave for 30 seconds.,
4: Add handful of raisins and stir.,
5: Microwave for 60 more seconds then stir.,
6: Top with banana slices.,
7: Sprinkle with cinnamon and drizzle with honey before serving.

Step 1 is completed when the dry oatmeal is on the bowl.
Step 2 is completed when there is water on the bowl.
Step 3 is completed when the bowl is on the microwave.
Step 4 is completed when there are raisins on the bowl.

And these are the possible states detected by the perception system and their probabilities:
oatmeal with water:0.54,
oatmeal with raisins with banana with cinnamon:0.06,
oatmeal with raisins:0.13,
oatmeal with raisins with banana:0.01,
oatmeal[dry]:0.15,
oatmeal with raisins with banana with cinnamon with honey:0.06,
cooked oatmeal:0.00,
empty:0.01 <</SYS>>
So, in which step of the recipe the user is?
[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.,
2: Add 1/2 cup water to the bowl.,
3: Microwave for 30 seconds.,
4: Add handful of raisins and stir.,
5: Microwave for 60 more seconds then stir.,
6: Top with banana slices.,
7: Sprinkle with cinnamon and drizzle with honey before serving.

Step 1 is completed when the dry oatmeal is on the bowl.
Step 2 is completed when there is water on the bowl.
Step 3 is completed when the bowl is on the microwave.
Step 4 is completed when there are raisins on the bowl.

And these are the possible states detected by the perception system and their probabilities:
oatmeal with water:0.54,
oatmeal with raisins with banana with cinnamon:0.06,
oatmeal with raisins:0.13,
oatmeal with raisins with banana:0.01,
oatmeal[dry]:0.15,
oatmeal with raisins with banana with cinnamon with honey:0.06,
cooked oatmeal:0.00,
empty:0.01 <</SYS>>
So, in which st

In [5]:
prompt = """
<s>[INST] <<SYS>>
Your are wrong, the user is on step 2 <</SYS>>
[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
Your are wrong, the user is on step 2 <</SYS>>
[/INST]
My apologies, you are correct. The user is currently on step 2. Thank you for correcting me!</s>


In [6]:
prompt = """
<s>[INST] <<SYS>>
Ok. <</SYS>>
So, what is the next step in the recipe?
[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
Ok. <</SYS>>
So, what is the next step in the recipe?
[/INST]
 Great, let's continue with the recipe! The next step is to cook the pasta in boiling water.

Instructions:

1. Add the pasta to a large pot of boiling, salted water.
2. Cook the pasta for 8-10 minutes, or until it's al dente.
3. Drain the pasta in a colander and rinse it with cold water to stop the cooking process.

Once the pasta is cooked, you can proceed to the next step in the recipe, which is to prepare the sauce. Do you have any questions or would you like me to provide more instructions?</s>


## 2. Few-Shot Prompting

In [2]:
import json

def read_json(file_paths):
    data = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            recipe_data = json.load(file)
            data.extend(recipe_data)
    return data

def seconds_to_minutes(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60

    return minutes, remaining_seconds

def two_digit_number(number):
    return f'{number:02}'

def make_hr_state(state):
    new_state = state.replace('+', ' with ')
    new_state = new_state.replace('[', '(')
    new_state = new_state.replace(']', ')')
    
    return new_state

def extract_data(data, include_step=True):
    dataset = []
    times = set()
    for entry in data:
        raw_seconds = int(entry['timestamp'])
        if raw_seconds in times:
            continue
        else:
            times.add(raw_seconds)
        minutes, seconds = seconds_to_minutes(raw_seconds)
        image_data_list = entry['objects']
        step = entry['step']
        
        for image_data in image_data_list:
            object = image_data['label']
            if 'state' in image_data:
                states = [f'{make_hr_state(s)}: {round(v,2)}' for s,v in image_data['state'].items()]
                text = f'At 01:{two_digit_number(minutes)}:{two_digit_number(seconds)} PM, the possible object states for {object} are: {", ".join(states)}. The predicted step is:'
                if include_step:
                    text = text + f' {step}'
                dataset.append(text)

    return dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained("/vast/work/public/ml-datasets/llama-2/Llama-2-13b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("/vast/work/public/ml-datasets/llama-2/Llama-2-13b-chat-hf", device_map="auto")
#tokenizer = AutoTokenizer.from_pretrained("./data/13b-quantized")
#model = AutoModelForCausalLM.from_pretrained("./data/13b-quantized", device_map="auto")

Loading checkpoint shards: 100%|██████████| 3/3 [01:27<00:00, 29.08s/it]
/ext3/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/ext3/miniconda3/envs/py38/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should 

In [4]:
file_paths = ['./perception_outputs/data_oatmeal_2023.11.03-18.51.03.json']
data = read_json(file_paths)
train_dataset = extract_data(data)
train_dataset[:10]

['At 01:00:14 PM, the possible object states for mug are: dripper with filter with coffee(dry): 0.47, dripper with filter: 0.3, tea-bag: 0.05, tea-bag with water: 0.15, dripper with filter with coffee(wet): 0.05, dripper(empty): 0.05, empty: 0.09. The predicted step is: 1',
 'At 01:00:15 PM, the possible object states for mug are: dripper with filter with coffee(dry): 0.61, dripper with filter: 0.01, tea-bag: 0.02, tea-bag with water: 0.06, dripper with filter with coffee(wet): 0.04, dripper(empty): 0.11, empty: 0.09, water: 0.06. The predicted step is: 1',
 'At 01:00:15 PM, the possible object states for bowl are: oatmeal(cooked): 0.55, oatmeal(dry): 0.45. The predicted step is: 1',
 'At 01:00:16 PM, the possible object states for mug are: dripper with filter with coffee(dry): 0.52, dripper with filter: 0.07, tea-bag: 0.12, tea-bag with water: 0.03, dripper with filter with coffee(wet): 0.07, dripper(empty): 0.06, empty: 0.05, water: 0.04, dripper with filter with coffee(drained): 0.1

In [5]:
prompt = """
<s>[INST] <<SYS>>
You are a reasoning system that has to identify which step of a recipe the user is on, given some object states identified by a perception system.<</SYS>>

Can you help me? I will give you an example. [/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
You are a reasoning system that has to identify which step of a recipe the user is on, given some object states identified by a perception system.<</SYS>>

Can you help me? I will give you an example. [/INST]
  Of course! I'd be happy to help. Please provide the recipe steps and the object states identified by the perception system, and I'll do my best to determine which step the user is on.

Here's an example to get started:

Recipe Steps:

1. Preheat the oven to 350°F (180°C).
2. Mix together flour, sugar, and butter to form a dough.
3. Roll out the dough into a circle.
4. Place the dough into a greased pie dish.
5. Fill the dough with a mixture of eggs, milk, and vanilla extract.
6. Bake for 25-30 minutes, or until the crust is golden brown and the filling is set.

Object States:

* The oven is on and preheating to 350°F (180°C).
* A mixing bowl contains flour, sugar, and butter.
* A rolling pin is on the counter.
* A greased pie dish is on the counte

In [6]:
prompt = f"""
<s>[INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
Step 1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.
Step 2: Add 1/2 cup water to the bowl.
Step 3: Microwave for 30 seconds.
Step 4: Add handful of raisins and stir.
Step 5: Microwave for 60 more seconds then stir.
Step 6: Top with banana slices.
Step 7: Sprinkle with cinnamon and drizzle with honey before serving.

Below you will see examples of object states identified by the perception system and the predicted steps for them.
{'. '.join(train_dataset[:10])}<</SYS>>
Are you ready to predict which step of a recipe the user is on, given some object states identified by a perception system?
[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
Step 1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.
Step 2: Add 1/2 cup water to the bowl.
Step 3: Microwave for 30 seconds.
Step 4: Add handful of raisins and stir.
Step 5: Microwave for 60 more seconds then stir.
Step 6: Top with banana slices.
Step 7: Sprinkle with cinnamon and drizzle with honey before serving.

Below you will see examples of object states identified by the perception system and the predicted steps for them.
At 01:00:14 PM, the possible object states for mug are: dripper with filter with coffee(dry): 0.47, dripper with filter: 0.3, tea-bag: 0.05, tea-bag with water: 0.15, dripper with filter with coffee(wet): 0.05, dripper(empty): 0.05, empty: 0.09. The predicted step is: 1. At 01:00:15 PM, the possible object states for mug are: dripper with filter with coffee(dry): 0.61, dripper with filter: 0.01, tea-bag: 0.02, tea-bag with water: 0.06, dripper with f

In [9]:
file_paths = ['./perception_outputs/data_oatmeal_2023.11.03-19.04.04.json']
data = read_json(file_paths)
test_dataset = extract_data(data, include_step=False)
test_dataset[:10]

['At 01:00:02 PM, the possible object states for bowl are: oatmeal with raisins: 0.21, oatmeal with water: 0.2, oatmeal(dry): 0.11, empty: 0.2, oatmeal with raisins with banana: 0.09, oatmeal(cooked): 0.73, oatmeal with raisins with banana with cinnamon: 0.09. The predicted step is:',
 'At 01:00:03 PM, the possible object states for bowl are: oatmeal with raisins: 0.29, oatmeal with water: 0.27, oatmeal(dry): 0.06, empty: 0.11, oatmeal with raisins with banana: 0.12, oatmeal(cooked): 0.37, oatmeal with raisins with banana with cinnamon: 0.04, oatmeal with raisins with banana with cinnamon with honey: 0.05. The predicted step is:',
 'At 01:00:03 PM, the possible object states for bowl are: empty: 0.73, oatmeal with raisins with banana with cinnamon with honey: 0.18, oatmeal with raisins with banana with cinnamon: 0.09. The predicted step is:',
 'At 01:00:04 PM, the possible object states for bowl are: oatmeal with raisins: 0.2, oatmeal with water: 0.19, oatmeal(dry): 0.03, empty: 0.55, 

In [12]:
prompt = f"""
<s>[INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
Step 1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.
Step 2: Add 1/2 cup water to the bowl.
Step 3: Microwave for 30 seconds.
Step 4: Add handful of raisins and stir.
Step 5: Microwave for 60 more seconds then stir.
Step 6: Top with banana slices.
Step 7: Sprinkle with cinnamon and drizzle with honey before serving.

Below you will see examples of object states identified by the perception system. Please predict the steps for them.
{test_dataset[1]}<</SYS>>

[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
Step 1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.
Step 2: Add 1/2 cup water to the bowl.
Step 3: Microwave for 30 seconds.
Step 4: Add handful of raisins and stir.
Step 5: Microwave for 60 more seconds then stir.
Step 6: Top with banana slices.
Step 7: Sprinkle with cinnamon and drizzle with honey before serving.

Below you will see examples of object states identified by the perception system. Please predict the steps for them.
At 01:00:03 PM, the possible object states for bowl are: oatmeal with raisins: 0.29, oatmeal with water: 0.27, oatmeal(dry): 0.06, empty: 0.11, oatmeal with raisins with banana: 0.12, oatmeal(cooked): 0.37, oatmeal with raisins with banana with cinnamon: 0.04, oatmeal with raisins with banana with cinnamon with honey: 0.05. The predicted step is:<</SYS>>

[/INST]
Based on the possible object states you provided, the predicted step is:

Step 4: Add 

In [14]:
prompt = f"""
<s>[INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
Step 1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.
Step 2: Add 1/2 cup water to the bowl.
Step 3: Microwave for 30 seconds.
Step 4: Add handful of raisins and stir.
Step 5: Microwave for 60 more seconds then stir.
Step 6: Top with banana slices.
Step 7: Sprinkle with cinnamon and drizzle with honey before serving.

Below you will see examples of object states identified by the perception system. Please predict the steps for them.
{test_dataset[10]}<</SYS>>

[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
It's an oatmeal recipe and these are the steps:
Step 1: Measure 4 Tablespoons of quick-cook oats and place in a microwave-safe bowl.
Step 2: Add 1/2 cup water to the bowl.
Step 3: Microwave for 30 seconds.
Step 4: Add handful of raisins and stir.
Step 5: Microwave for 60 more seconds then stir.
Step 6: Top with banana slices.
Step 7: Sprinkle with cinnamon and drizzle with honey before serving.

Below you will see examples of object states identified by the perception system. Please predict the steps for them.
At 01:00:06 PM, the possible object states for bowl are: empty: 0.72, oatmeal with raisins with banana with cinnamon with honey: 0.0, oatmeal with raisins with banana with cinnamon: 0.02, oatmeal(cooked): 0.19, oatmeal with water: 0.06, oatmeal with raisins with banana: 0.0, oatmeal(dry): 0.0, oatmeal with raisins: 0.0. The predicted step is:<</SYS>>

[/INST]
Based on the possible object states you provided, the predicted step is:

Step 3: Microwav

#### Free-form Questions

In [17]:
prompt = f"""
<s>[INST] <<SYS>>
When did the user start the step 1 of the recipe?<</SYS>>
[/INST]
"""
prompt_template=f'{prompt}'

print("Output:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

Output:
<s> 
<s> [INST] <<SYS>>
When did the user start the step 1 of the recipe?<</SYS>>
[/INST]
 As a system, I don't have the ability to track the user's actions or start times. The user may have started the step 1 of the recipe at any point in the past, and there is no information available to determine when exactly they started.

However, if the user has provided any information or context about their progress in the recipe, it may be possible to infer when they started step 1 based on that information. For example, if the user has mentioned that they started the recipe yesterday, it may be reasonable to assume that they started step 1 on the previous day.

Without any additional information, it is not possible to determine when the user started step 1 of the recipe with certainty.</s>
